<a href="https://colab.research.google.com/github/ReaganJHarris/ET_Retrieval/blob/main/ET_Retrieval_Loop_Ls5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries and authorizations

In [ ]:
import os
import sys
import datetime
from time import strftime, strptime, gmtime
import math
import pprint
import copy
import pandas as pd
import numpy as np


In [ ]:
#@title Install Earthengine Python API

!pip install 'pyOpenSSL>=0.11'
!pip install earthengine-api

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 3.6 MB 23.2 MB/s 


In [ ]:
#@title authenticate and initialize the Earthengine API
!earthengine authenticate

import ee
ee.Initialize()

image = ee.Image('srtm90_v4')
print(image.getInfo())

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=_pku8AFfNKfBYsBoJyqKbGy2eCcOWQa-T4uE6a_FjKc&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi7BdUBKd8Z4MuQSeC92AjFciZ2iS9CCUa5meM8mZjIAQXzReiOKZM

Successfully saved authorization token.
{'type': 'Image', 'bands': [{'id': 'elevation', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [432000, 144000], 'crs

In [ ]:
#@title mount Google Drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
#@title authorize Google Drive
from google.colab import auth
auth.authenticate_user()

# Constants

## Datasets

In [ ]:
# Datasets

Counties = ee.FeatureCollection("TIGER/2018/Counties")
DayMET = ee.ImageCollection("NASA/ORNL/DAYMET_V3")
SRTM = ee.Image("USGS/SRTMGL1_003")
MODIS = ee.ImageCollection("MODIS/006/MOD11A2")
GLDAS = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")
ARD_LST_5 = ee.ImageCollection('projects/ColoradoView/Landsat/ARD/5')
MODIS_Albedo = ee.ImageCollection("MODIS/006/MCD43A3")
ERA5 = ee.ImageCollection("ECMWF/ERA5/DAILY")
GRIDMET = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")
ARD_ET_5 = ee.ImageCollection('projects/ColoradoView/Landsat/ARD_ET/5')

## ARD Mask

In [ ]:
#@title Mask fuctions
def maskLandsatARD(image):
    fillBitMask = 1
    clearBitMask = 1 << 1
    cloudBitMask = 1 << 5
  
    lst_name = 'b1'
    pixelqa_name = 'b2'
    lstqa_name = 'b3'
    lstqa_scale_name = 'STQA_scale_factor'
  
    lstqa_threshold = 5.0
  
    # get the pixel QA band
    # pixelqa_name: 'b2'
    pixelqa = image.select(pixelqa_name)
  
    # get the STQA band
    lstqa = image.select(lstqa_name)
    stqa_scale = ee.Number(image.get(lstqa_scale_name))
    
    mask = pixelqa.bitwiseAnd(fillBitMask).eq(0) \
      .And(pixelqa.bitwiseAnd(clearBitMask).neq(0)) \
      .And(pixelqa.bitwiseAnd(cloudBitMask).eq(0)) \
      .And(lstqa.multiply(stqa_scale).lt(lstqa_threshold))
        
    #.and(pixelqa.bitwiseAnd(cloudConfBit7Mask).eq(0))
    #.and(pixelqa.bitwiseAnd(cirrusConfBit9Mask).eq(0))
          
          
    # Return the masked image (excluding the PIXELQA and STQA layers)
    return image.updateMask(mask) \
        .select(lst_name)  # [lst_name, pixelqa_name, lstqa_name]

def toKelvin(image):
    Celsius = image.get(['tmax', 'tmin'])
    Kelvin = image.add(273)
    return Kelvin

def  maskAlb_swr(image):
    QualityBitMask = 0 << 1
    
    pixelqa = image.select('BRDF_Albedo_Band_Mandatory_Quality_shortwave')
    
    
    mask = pixelqa.bitwiseAnd(QualityBitMask).eq(0)
          
          
    return image.updateMask(mask) \
        .select(['Albedo_BSA_shortwave']) \
        .multiply(0.001) \
        .addBands(mask.rename('mask'))


# Proccessing dates

## Vaild Dates and c parameter function

In [ ]:
# dates


# for key, values in GEOID.items():
#def FIPS_valid_dates(county):
def parameter_fn(county_dict, TgtCounty_GEOID, start_date_str):

  #start_date_str = '2017-01-01'
  #end_date_str = '2017-12-31'

  Date_st = ee.Date(start_date_str)
  #Date_ed = ee.Date(end_date_str)
  Date_ed = Date_st.advance(1, 'year')

  date_range = Date_ed.difference(Date_st, 'day').getInfo()

  # get the State boundary of Colorado
  # TgtCounty_index = 0

  #TgtCounty_GEOID = countyGEOID_array #[TgtCounty_index] #08123：	Weld 
  # look up county name in the dict using the key (GEOID)
  if (TgtCounty_GEOID in county_dict):
    #TgtCounty_Name = countyName_array #[TgtCounty_index]
    TgtCounty_Name = county_dict[TgtCounty_GEOID]
  else:
    # GEOID not found
    raise ValueError(f"Error: given GEOID {TgtCounty_GEOID} not found.")


  TgtCounty_Bnds = Counties.filter(ee.Filter.eq('GEOID', TgtCounty_GEOID))
  # print(TgtCounty_Bnds)
  # print(TgtCounty_Bnds.first().geometry().getInfo())
  TgtCounty_Bnds_geom = TgtCounty_Bnds.first().geometry()
  TgtCounty_centroid = TgtCounty_Bnds_geom.centroid()
  
  geom_State_Bnds_CO_geometries = TgtCounty_Bnds.first().geometry().getInfo()
  # print(geom_State_Bnds_CO_geometries)
  # if isinstance(geom_State_Bnds_CO_geometries, list):
  if geom_State_Bnds_CO_geometries['type'] == 'GeometryCollection':
    for igeom in geom_State_Bnds_CO_geometries['geometries']:
      if igeom['type'] == 'Polygon' or igeom['type'] == 'MultiPolygon':
        geom_State_Bnds_CO = igeom['coordinates']
        break
  else:
    geom_State_Bnds_CO = geom_State_Bnds_CO_geometries['coordinates']

  # print(geom_State_Bnds_CO)
        
        
  # geom_State_Bnds_CO = TgtCounty_Bnds.first().geometry().getInfo()['coordinates'] 
  

  geom_str = str(geom_State_Bnds_CO)
                
  total_count_TgtCounty_500 = ee.Image(1).clipToCollection(TgtCounty_Bnds).rename('count').reduceRegion( \
        reducer= ee.Reducer.count(), \
        geometry= TgtCounty_Bnds.first().geometry(), \
        scale= 500, \
        maxPixels= 1e13 \
      ).get('count')

  ## valid dates function
  date_range = Date_ed.difference(Date_st, 'day').getInfo();

  # create list of dates from start through date range
  days = ee.List.sequence(0, date_range, 1.0)
  def generate_date_feature(d):
  # date_Fts = days.map(function(d) {
    #  d: the index of the continuous sequence (i.e., days)
    #  cur_date: ee.Date of the day pointed by d
    cur_date = Date_st.advance(d, 'day')
    #  filter by the cur_date
    cur_ImgCol = ARD_ET_5.filterDate(cur_date)
    #  reduce resolution of the ImageCollection 
    # cur_ImgCol = cur_ImgCol.map(function(inImg){
    def coarse_reproject(inImg):
      # reproject in very low resolution (1920m) in order to reduce the calulation burdun.
      return inImg.reproject('EPSG:3857', None, 500)
    
    cur_ImgCol = cur_ImgCol.map(coarse_reproject)
    # mosaic ImageCollection and clip to the target boundary
    cur_ImgMosaic = ee.Image(cur_ImgCol.mosaic()).clipToCollection(TgtCounty_Bnds)
    # get the number of bands in the Image
    num_bands = cur_ImgMosaic.bandNames().size()
    # If number of bands > 0, return a normal date string, otherwise get an empty sting
    cur_date_str = ee.Algorithms.If(num_bands.gt(0), cur_date.format('YYYY-MM-dd'), '')

    cur_countMaskVal = ee.Algorithms.If(num_bands.gt(0), 
      cur_ImgMosaic.mask().clipToCollection(TgtCounty_Bnds).select(0).updateMask(cur_ImgMosaic.mask().select(0)).rename('count').reduceRegion( 
        reducer= ee.Reducer.count(), 
        geometry= TgtCounty_Bnds.first().geometry(), 
        scale= 500, 
        maxPixels= 1e13 
      ).get('count'), 
      0)
    
    # create a empty Feature with two properties: date_str and date_str_len
    ret_date_Ft = ee.Feature(None, { 
    'date_str': cur_date_str, 
    'date_str_len': ee.String(cur_date_str).length(), 
    'count': cur_countMaskVal})
    return ret_date_Ft

  date_Fts = days.map(generate_date_feature)
  # print("date_Fts", date_Fts)

  # filter List of Features to exclude those invalid dates (i.e., date_str_len == 0)
  valid_dates_filtered = date_Fts \
    .filter(ee.Filter.gt('date_str_len', 0)) \
    .filter(ee.Filter.gt('count', ee.Number(total_count_TgtCounty_500).multiply(0.5)))
  # print("valid_dates_Fts", valid_dates_filtered)

  # convert the List of Features to List of Strings (i.e., list of date_str_len)
  def features_to_string(inFt):
    inFt = ee.Feature(inFt)
    return inFt.get('date_str')

  valid_dates_filtered_StringList = valid_dates_filtered.map(features_to_string)
  # batch fetch List content to client
  valid_dates = valid_dates_filtered_StringList.getInfo()
  num_valid_dates = len(valid_dates)
  print(num_valid_dates)
  # print("valid_dates", num_valid_dates, valid_dates)
  #return valid_dates

  #Define loop to derive c parameter via month in valid date

  #Import dictionary(?)/text file with c parameters by month
  #Moffat monthly averages in 2003
  from datetime import datetime
  # TODO use dataframe...
  CO_cPara_DF = pd.read_pickle('/gdrive/MyDrive/Ls5_Elbert_cPara.pkl')
  SeasonalCO_cPara1 = CO_cPara_DF.groupby(['Year','Season']).agg({'mean':['mean']}).reset_index()
  # SeasonalCO_cPara.groupby('Season').agg({'mean mean': ['mean']}).reset_index()
  SeasonalCO_cPara1.columns = [' '.join(col).strip() for col in SeasonalCO_cPara1.columns.values]
  SeasonalCO_cPara = SeasonalCO_cPara1.groupby('Season').agg({'mean mean': ['mean']}).reset_index()
  SeasonalCO_cPara.columns = [' '.join(col).strip() for col in SeasonalCO_cPara.columns.values]
  SeasonalCO_cPara = SeasonalCO_cPara.rename(columns={'mean mean mean':'mean'})
  SeasonalCO_cPara = SeasonalCO_cPara.set_index('Season')
  Seasonal_cPara = SeasonalCO_cPara.to_dict('dict')

  cPara_Arr = []

  for x in valid_dates:
    dt_conversion = datetime.strptime(x, '%Y-%m-%d')
    month_date = dt_conversion.month
    # if 1 <= month_date <= 3:
    #   season = 'Winter'
    #   cPara = Seasonal_cPara['mean'][season]
    #   continue
    if 3 < month_date <= 6:
      season = 'Spring'
      cPara = Seasonal_cPara['mean'][season]
    elif 6 < month_date <= 9:
      season = 'Summer'
      cPara = Seasonal_cPara['mean'][season]
    else:
      season = 'Fall'  
      cPara = Seasonal_cPara['mean'][season]
      # print(cPara)
      # print(type(cPara))
    cPara_Arr.append(cPara)


  return valid_dates, TgtCounty_Bnds, TgtCounty_Name, cPara_Arr


# Potential ET function

In [ ]:
# @title Potential ET function
# Potential ET from FAO
#def potential_ET_fn(in_date):
def potential_ET_fn(in_date, TgtCounty_Bnds):
# in_date = FIPS_valid_dates(GEOID)
  Date_st = in_date
  Date_ed = Date_st.advance(1,'day')

  # Potential ET

  ARD_Img = ee.Image(ARD_LST_5 \
    .filterDate(Date_st.format('yyyy-MM-dd'), Date_ed.format('yyyy-MM-dd')) \
    .map(maskLandsatARD) \
    .mosaic() \
    )
      
  ARD_Img = ARD_Img.select(0).multiply(0.1).rename('LST')

  #ARD mask for other image collections
  ARD_Img_mask = ARD_Img.mask().clip(TgtCounty_Bnds)

  # DayMET
  DayMET_ImgCol = DayMET \
    .filterBounds(TgtCounty_Bnds) \
    .filterDate(Date_st.format('yyyy-MM-dd'), Date_ed.format('yyyy-MM-dd')) \
    .select(['tmax', 'tmin', 'srad', 'dayl'])

  DayMET_Img = ee.Image(DayMET_ImgCol \
    .map(toKelvin) \
    .mosaic()) \
    .clip(TgtCounty_Bnds) \
    .updateMask(ARD_Img_mask)

  #Gridmet
  GRIDMET_ImgCol = GRIDMET \
    .filterDate(Date_st.format('yyyy-MM-dd'), Date_ed.format('yyyy-MM-dd')) \
    .filterBounds(TgtCounty_Bnds) \
    .select('eto', 'etr', 'vpd', 'vs', 'srad', 'rmax', 'rmin')
    
  GRIDMET_Img = ee.Image(GRIDMET_ImgCol \
    .mosaic()) \
    .clipToCollection(TgtCounty_Bnds)

  #ERA5
  ERA5_ImgCol = ERA5 \
    .filterBounds(TgtCounty_Bnds) \
    .filterDate(Date_st.format('yyyy-MM-dd'), Date_ed.format('yyyy-MM-dd')) \
    .select(['u_component_of_wind_10m', 'v_component_of_wind_10m']) 
        
  ERA5_Img = ee.Image(ERA5_ImgCol \
    .mosaic()) \
    .resample('bilinear') \
    .clipToCollection(TgtCounty_Bnds) \
    .updateMask(ARD_Img_mask)

  # MODIS Albedo
  MOD_Alb_ImgCol = MODIS_Albedo \
    .filterBounds(TgtCounty_Bnds) \
    .filterDate(Date_st.format('yyyy-MM-dd'), Date_ed.format('yyyy-MM-dd'))

  Alb_Img = ee.Image(MOD_Alb_ImgCol \
    .map(maskAlb_swr) \
    .mosaic()) \
    .resample('bilinear') \
    .clipToCollection(TgtCounty_Bnds) \
    .updateMask(ARD_Img_mask)

  # GLDAS
  # GLDAS_ImgCol = GLDAS \
  #   .filterBounds(TgtCounty_Bnds) \
  #   .filterDate(Date_st.advance(6, 'hours'), Date_ed.advance(6, 'hours')) \
  #   .select(['Swnet_tavg', 'PotEvap_tavg', 'Lwnet_tavg', 'SWdown_f_tavg'])
      
  # GLDAS_Img = GLDAS_ImgCol \
  #   .reduce(ee.Reducer.mean()) \
  #   .resample('bilinear') \
  #   .clipToCollection(TgtCounty_Bnds) \
  #   .updateMask(ARD_Img_mask)

  #  Es minimum temp
  # net_swr_Swnet = DayMET_Img.expression('((1-alb)*swr*day_seconds)/(1.0e6)', {
  #   'swr': GRIDMET_Img.select('srad'),
  #   'alb': Alb_Img.select('Albedo_BSA_shortwave'),
  #   'day_seconds': 86400
  # })  \
  #   .rename('Net_Daily_SWR');
  SRTM_Img = SRTM \
    .select('elevation') \
    .clipToCollection(TgtCounty_Bnds) \
  
  current_year = Date_st.get('year');
  empty_date = ee.Date.fromYMD(current_year, 1, 1);
  # print(empty_date);
  DOY_st =  Date_st.difference(empty_date, 'day').add(1)
  # print(DOY_st);
  latitude = ee.Image.pixelLonLat().select('latitude').multiply(ee.Number(math.pi).divide(180));
  # Map.addLayer(latitude, {}, 'Latitude');
  earth_sun_Dist = DayMET_Img.expression('1 + 0.033 * cos((2 * pi)/365 * DOY)', {                                                      
      'pi': math.pi,  \
      'DOY': DOY_st,  \
    });
  # print(earth_sun_Dist);
  solar_constant = 0.0820 
  # MJ/m2/min
  solar_decl = DayMET_Img.expression('0.409 * sin(2*pi/365 * DOY - 1.39)',  { 
      'pi': math.pi,  \
      'DOY': DOY_st,  \
    });
  # print(delta);
  # gamma = (latitude.tan().multiply(delta.tan()).multiply(-1)).acos();
  # sunset_hr_angle = math.acos(math.tan(latitude) * (math.tan(delta)) * -1)
  sunset_hr_angle = DayMET_Img.expression('acos(tan(latitude) * tan(delta) * -1)', {
      'latitude': latitude, \
      'delta': solar_decl \
  })
  extraterrestial_rad = DayMET_Img.expression(  \
      '((24 * 60) / pi) * Gsc * dr * (gamma * sin(latitude) * sin(delta) + cos(latitude) * cos(delta) * sin(gamma))', {
          'pi': math.pi,  \
          'Gsc': solar_constant,  \
          'dr': earth_sun_Dist, \
          'gamma': sunset_hr_angle, \
          'latitude': latitude, \
          'delta': solar_decl, \
        });
  # print(extraterrestial_rad.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100));

  solar_rad = extraterrestial_rad.expression('(0.75 + 2 * 1.0e-5 * elev) * Rs', {
      'elev': SRTM_Img, \
      'Rs': extraterrestial_rad \
    });

  net_swr_Swnet = solar_rad.expression('(1-alb)*Rsa', {
    'alb': Alb_Img.select('Albedo_BSA_shortwave'),  \
    'Rsa': solar_rad  \
  });

  # print('Net shortwave rad Senay 2013 Eq. 8', net_swr_Swnet.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100));

  # swr_mean = net_swr_Swnet.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('shortwave rad', swr_mean)

  #  GRIDMET Actual Vapor Pressure
  #  Es minimum temp
  svp_min = DayMET_Img.expression('0.6108*(e**((17.27*tmin)/(tmin+237.3)))', {
      'e': 2.71828,
      'tmin': DayMET_Img.select('tmin').subtract(273.16)
  }) \
    .rename('Saturated_Vapor_Pressure');

  # svp_min_mean = svp_min.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('Sat_Vapor_Press_min_mean', svp_min_mean)

  # Es Maximum Temp
  svp_max = DayMET_Img.expression('0.6108*(e**((17.27*tmax)/(tmax+237.3)))', {
    'e': 2.71828,
    'tmax': DayMET_Img.select('tmax').subtract(273.15)
  }) \
    .rename('Saturated_Vapor_Pressure')

  # svp_max_mean = svp_max.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('Sat_Vapor_Press_max_mean', svp_max_mean);

  # Actual Vapor Pressure from relative humidity
  Ea = ((svp_min.multiply(GRIDMET_Img.select('rmax').divide(100)))
    .add(svp_max.multiply(GRIDMET_Img.select('rmin').divide(100)))) \
    .divide(2) \
    .rename('Actual_Vapor_Pressure');

  # AVP_mean = Ea.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('ActualVaporPressure_mean', AVP_mean)

  # Net Long-wave Radiation
  net_lwr = DayMET_Img.expression('(sigma*((tmin**4+tmax**4)/2))*(0.34-(0.14*Kpa))*(1.35*(Rs_Rso)-0.35)', { \
    'Rs_Rso': 1, \
    'sigma': 0.0000000049, \
    'tmin': DayMET_Img.select('tmin'), \
    'tmax': DayMET_Img.select('tmax'), \
    'Kpa': Ea.sqrt() \
  }) \
    .rename('Net_longwave_radiation')
  # lwr_mean = net_lwr.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('longwave mean', lwr_mean)


  # Calculating Rn = R_ns - R_nl
  # This output is in MJ/m^2/d
  ClearSky_Rn = net_swr_Swnet.subtract(net_lwr) \
    .rename('Clear_Sky_Net_Radiation')
  # net_rad_mean = ClearSky_Rn.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('ClearSky net rad', net_rad_mean)

  # AVP_mean = Ea.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100);
  # print('ActualVaporPressure_mean', AVP_mean);

  # Es average of saturated min and max VP
  Es = (svp_min.add(svp_max)).divide(2).rename('saturated_vp')
  # Es_mean = Es.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('Es mean', Es_mean)
  # saturation vapor pressure deficit (VPD) (kPa)
  Es_Ea = Es.subtract(Ea).rename('VPD')
  # VPD_mean = Es_Ea.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('VPD_mean', VPD_mean)

  # Delta is the Slope of the saturation vapor pressure curve [kPa/C] 
  # FAO Irrigation and Drainage Paper 56, (Page 37)
  Tmin_C = DayMET_Img.select('tmin').subtract(273)
  Tmax_C = DayMET_Img.select('tmax').subtract(273)
  Tavg_C = (Tmax_C.add(Tmin_C)).divide(2).rename('Tavg')

  delta = DayMET_Img.expression('(4098*(Es))/(T+237.3)**2', {
    'Es': Es,
    'T': Tavg_C,
  }) \
    .rename('delta');
  # delta_mean = delta.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('delta_mean', delta_mean)

  # P = 101.3((293-0.0065*Z)/293)^5.25
  # Z = elevation (m)
  # SRTM_Img = SRTM \
  #   .select('elevation') \
  #   .clipToCollection(TgtCounty_Bnds) \

  # pressure
  pressure = SRTM_Img.expression('101.3*((293-0.0065*SRTM_Img)/293)**5.25', { \
    'SRTM_Img': SRTM_Img \
  }) \
    .rename('Pressure')
  # pressure_mean = pressure.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('pressure_mean', pressure_mean)

  # psychrometric constant, Gamma 
  # Canstants for the equation:
  lambda_1 = 2.45 # latent heat of vapoization (MJ/kg)
  specific_heat = 1.013e-3 # specific heat at constant pressure (MJ/kg/C)
  epsilon = 0.622 # ratio molecular weight of water vapour/dry air (unitless)
  gamma = DayMET_Img.expression('(Cp*pressure)/(epsilon*lambda)', { \
    'Cp': specific_heat, \
    'pressure': pressure, \
    'epsilon': epsilon, \
    'lambda': lambda_1 \
  })
  # gamma_mean = gamma.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('gamma mean', gamma_mean)

  # Wind speed from ERA5
  # sqrt(u^2+v^2)

  # u wind is longitude
  u_wind = ERA5_Img.select('u_component_of_wind_10m')

  # v wind is latitude
  v_wind = ERA5_Img.select('v_component_of_wind_10m')

  # u2 wind speed sqrt(u^2+v^2)
  # 
  wind_spd_10m = (u_wind.pow(2).add(v_wind.pow(2))).sqrt()

  #mConverting 10m wind speed to 2m
  # wind_conversion_facotr = 4.87 / ln(67.8 (10) - 5.42) = 0.75
  wind_spd_2m = wind_spd_10m.multiply(0.75).rename('wind_speed_2m');
  # wind_speed_mean = wind_spd_2m.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
  # print('wind_speed', wind_speed_mean)

  # ETo equation
  ETo = DayMET_Img.expression('(0.408*delta*(Rn-G)+gamma*(900/(T+273))*u2*(Es_Ea)) / (delta+gamma*(1+0.34*(u2)))', { \
      'delta': delta, \
      'Rn': ClearSky_Rn, \
      'G': 0, \
      'gamma': gamma, \
      'T': Tavg_C, \
      'u2': wind_spd_2m, \
      'Es_Ea': Es_Ea \
  })
  # print(ETo.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100))
  return ETo

# Actual ET Function

In [ ]:
#@title Actual ET function

def Actual_ET_Fn(in_date, PET_value, TgtCounty_Bnds, cPara):
  
  Date_st = in_date
  Date_ed = Date_st.advance(1,'day')

  #  ARD LST Landsat 
  ARD_Img = ee.Image( \
    ARD_LST_5 \
    .filterDate(Date_st.format('yyyy-MM-dd'), Date_ed.format('yyyy-MM-dd')) \
    .map(maskLandsatARD) \
    .mosaic() \
    )
    
  ARD_Img = ARD_Img.select(0).multiply(0.1).rename('LST')

  ARD_Img_mask = ARD_Img.mask().clip(TgtCounty_Bnds)

  # DayMET

  DayMET_ImgCol = DayMET \
    .filterBounds(TgtCounty_Bnds) \
    .filterDate(Date_st.format('yyyy-MM-dd'), Date_ed.format('yyyy-MM-dd')) \
    .select(['tmax', 'tmin', 'srad', 'dayl'])

  DayMET_Img = ee.Image(DayMET_ImgCol \
    .map(toKelvin) \
    .mosaic()) \
    .clip(TgtCounty_Bnds) \
    .updateMask(ARD_Img_mask)

  GRIDMET_ImgCol = GRIDMET \
    .filterDate(Date_st.format('yyyy-MM-dd'), Date_ed.format('yyyy-MM-dd')) \
    .filterBounds(TgtCounty_Bnds) \
    .select('eto', 'etr', 'vpd', 'vs', 'srad', 'rmax', 'rmin')
  
  GRIDMET_Img = ee.Image(GRIDMET_ImgCol \
    .mosaic()) \
    .clipToCollection(TgtCounty_Bnds)

  # c_coef = cPara
  
  # Cold Reference Pixel
  Temp_Cold = DayMET_Img \
    .select('tmax') \
    .multiply(cPara) \
    .rename('Reference Cold Pixel')

  Temp_min = DayMET_Img \
    .select('tmin') 

  Temp_mean = (DayMET_Img.select('tmax').add(Temp_min)).divide(2) \
    .rename('T_avg')
  
  # Calculate Air Density
  Tkv = Temp_mean.multiply(1.01) \
    .rename('Virtual_Temperature')

  SRTM_Img = SRTM.select('elevation') \
    .clipToCollection(TgtCounty_Bnds)

  pressure = SRTM_Img.expression('101.3*((293-0.0065*SRTM_Img)/293)**5.25', { \
    'SRTM_Img': SRTM_Img \
  }) \
    .rename('Pressure')

  Air_Density = pressure.expression('3.486*(pressure/Tkv)', { \
    'pressure': pressure, \
    'Tkv': Tkv \
  }) \
    .rename('Air_Density')
  
  # MODIS Albedo
  MOD_Alb_ImgCol = MODIS_Albedo \
    .filterBounds(TgtCounty_Bnds) \
    .filterDate(Date_st.format('yyyy-MM-dd'), Date_ed.format('yyyy-MM-dd'))

  Alb_Img = ee.Image(MOD_Alb_ImgCol \
    .map(maskAlb_swr) \
    .mosaic()) \
    .resample('bilinear') \
    .clipToCollection(TgtCounty_Bnds) \
    .updateMask(ARD_Img_mask)

  # Shortwave radiation

  # SW_down = DayMET_Img.select('SWdown_f_tavg_mean') \
  #   .multiply(86400)

  # for srad from GRIDMET
  # net_swr_Swnet = Alb_Img.expression('((1-alb)*swr*day_seconds)/(1.0e6)', {
  #     'swr': GRIDMET_Img.select('srad'),
  #     'alb': Alb_Img.select('Albedo_BSA_shortwave'),
  #     'day_seconds': 86400
  # }) \
  #   .rename('Net_Daily_SWR');

  # ---Shortwave radiation from Senay 2013----
  current_year = Date_st.get('year');
  empty_date = ee.Date.fromYMD(current_year, 1, 1);
  # print(empty_date);
  DOY_st =  Date_st.difference(empty_date, 'day').add(1);
  # print(DOY_st);
  latitude = ee.Image.pixelLonLat().select('latitude').multiply(ee.Number(math.pi).divide(180));

  earth_sun_Dist = ee.Number.expression('1 + 0.033 * cos((2 * pi)/365 * DOY)', {                                                           
      'pi': math.pi,  \
      'DOY': DOY_st \
    });
  # print(earth_sun_Dist);
  solar_constant = 0.0820 
  # MJ/m2/min
  # Solar declination
  solar_decl = DayMET_Img.expression('0.409 * sin(2*pi/365 * DOY - 1.39)',  {
      'pi': math.pi,  \
      'DOY': DOY_st  \
    });
  # print(delta);
  # gamma = (latitude.tan().multiply(delta.tan()).multiply(-1)).acos();
  # gamma = math.acos(math.tan(latitude) * (math.tan(delta)) * -1)

  # sunset hour angle
  sunset_hr_angle = DayMET_Img.expression('acos(tan(latitude) * tan(delta) * -1)', {
      'latitude': latitude, \
      'delta': solar_decl \
  })

  # extraterrestial radiation
  extraterrestial_rad = DayMET_Img.expression(  \
      '((24 * 60) / pi) * Gsc * dr * (gamma * sin(latitude) * sin(delta) + cos(latitude) * cos(delta) * sin(gamma))', {
          'pi': math.pi,  \
          'Gsc': solar_constant,  \
          'dr': earth_sun_Dist, \
          'gamma': sunset_hr_angle, \
          'latitude': latitude, \
          'delta': solar_decl, \
        });
  # print(extraterrestial_rad.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100));

  # Solar radiation
  solar_rad = extraterrestial_rad.expression('(0.75 + 2 * 1.0e-5 * elev) * Rs', { \
      'elev': SRTM_Img, \
      'Rs': extraterrestial_rad \
    });

  # net shortwave/solar radiation
  net_swr_Swnet = solar_rad.expression('(1-alb)*Rsa', { \
    'alb': Alb_Img.select('Albedo_BSA_shortwave'),  \
    'Rsa': solar_rad  \
  });

  # print('Net shortwave rad Senay 2013 Eq. 8', net_swr_Swnet.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100));

  #  GRIDMET Actual Vapor Pressure
  #  Es minimum temp
  svp_min = DayMET_Img.expression('0.6108*(e**((17.27*tmin)/(tmin+237.3)))', {
      'e': 2.71828,
      'tmin': DayMET_Img.select('tmin').subtract(273.16)
  }) \
    .rename('Saturated_Vapor_Pressure');
  
  # Es Maximum Temp
  svp_max = DayMET_Img.expression('0.6108*(e**((17.27*tmax)/(tmax+237.3)))', {
    'e': 2.71828,
    'tmax': DayMET_Img.select('tmax').subtract(273.16)
  }) \
    .rename('Saturated_Vapor_Pressure')
  
  # Actual Vapor Pressure from relative humidity
  Ea = ((svp_min.multiply(GRIDMET_Img.select('rmax').divide(100)))
    .add(svp_max.multiply(GRIDMET_Img.select('rmin').divide(100)))) \
    .divide(2) \
    .rename('Actual_Vapor_Pressure');

  # Net Long-wave Radiation
  net_lwr = DayMET_Img.expression('(sigma*((tmin**4+tmax**4)/2))*(0.34-(0.14*Kpa))*(1.35*(Rs_Rso)-0.35)', { \
    'Rs_Rso': 1, \
    'sigma': 0.0000000049, \
    'tmin': DayMET_Img.select('tmin'), \
    'tmax': DayMET_Img.select('tmax'), \
    'Kpa': Ea.sqrt() \
  }) \
    .rename('Net_longwave_radiation')

  # Calculating Rn = R_ns - R_nl
  # This output is in MJ/m^2/d
  ClearSky_Rn = net_swr_Swnet.subtract(net_lwr) \
    .rename('Clear_Sky_Net_Radiation')

  # Temperature Difference (dT) = ((Rn*Rah)/(Pa*Cp))
  Temp_diff = DayMET_Img.expression('(Rn*Rah)/(Pa*Cp)', { \
    'Rn': ClearSky_Rn.divide(86.4), \
    'Rah': 110, \
    'Pa': Air_Density, \
    'Cp': 1.013 \
  }) \
    .rename('Temperature_Difference')

  # Hot/Dry limit (Th)
  Hot_pixel_ref = Temp_Cold.add(Temp_diff) \
    .rename('Hot_pixel_ref')

  # Fractional Evapotranspiration
  # ETf = (Th-Ts)/(dT) 
  # frac_evap = (Hot_pixel_ref.subtract(ARD_Img.select('LST'))) \
  #   .divide(Temp_diff) \
  #   .rename('Fractional_Evapotranspiration')
  frac_ET = (Hot_pixel_ref.subtract(ARD_Img.select('LST'))) \
  .divide(Temp_diff)  \
  .rename('Fractional_ET')

  # Map.addLayer(frac_ET, {}, 'ETf');

  # ETf_mean = frac_ET.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100);
  # print('ETf_mean', ETf_mean);

  ETf_gte1d05 = frac_ET.gte(1.05).cast({'Fractional_ET':'float'});
  ETf_lt0 = frac_ET.lt(0).cast({'Fractional_ET':'float'});

  # print('ETf_gte1d05',ETf_gte1d05);

  ETf_cap = ETf_gte1d05.multiply(1.05).add(ee.Image(1.0).subtract(ETf_gte1d05).multiply(frac_ET));
  ETf_cap = ETf_lt0.multiply(0).add(ee.Image(1.0).subtract(ETf_lt0).multiply(ETf_cap));
  # ETf_cap_mean = ETf_cap.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100);
  # print('ETf_cap_mean', ETf_cap_mean);
  
  ETo_FAO = PET_value

  # Actual Evapotranspiration
  # ETa = ETf * k * ETo
  actual_ET = DayMET_Img.expression('ETf*k*ETo', { 
    'ETf': ETf_cap,
    'k': 1.0,
    'ETo': ETo_FAO
  }) \
    .rename('Actual_Evapotranspiration')
  
  mask1 = actual_ET.gte(0);

  ETa_positive = actual_ET.updateMask(mask1);

  # print(ETa_positive.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100))

  return ETa_positive

# Export and call wrapper functions

## Export images to GEE asset folder

In [ ]:
def export_to_asset(image, desc, assetID, TgtCounty_Bnds):

  task_asset = ee.batch.Export.image.toAsset(
    image= image,
    description= desc,
    assetId= assetID, # valid_dates, #date, county
    region= TgtCounty_Bnds.first().geometry(),
    scale= 30,
    maxPixels= 1e13
  )

  task_asset.start()


## Function to run PET and ETa functions and export function

In [ ]:

def ET_retrieval_county_fn(valid_dates, TgtCounty_Bnds, TgtCounty_Name, cPara_coefArr):

  for i, c_Coef in zip(valid_dates, cPara_coefArr):
    day = ee.Date(i)
    PET_value = potential_ET_fn(day, TgtCounty_Bnds)
    ETa = Actual_ET_Fn(day, PET_value, TgtCounty_Bnds, c_Coef)

    Landsat_folder = 'Ls5'
    desc = TgtCounty_Name + '_' + i

    assetID= 'projects/ColoradoView/ET_Seasonal_cPara/' + TgtCounty_Name + '/' + Landsat_folder + '/' + i
    export_to_asset(ETa, desc, assetID, TgtCounty_Bnds)


## Function to loop through defined counties and years and calling export function

In [ ]:
def ET_retrieval_wrapper(county_dict, in_year):
  GEOID = copy.deepcopy(county_dict)

  TgtCounty_GEOID = list(GEOID.keys())[0]
  start_date_str = str(in_year) + '-01-01' #'2013-01-01'


  for TgtCounty_GEOID in GEOID.keys():
    #valid_dates, TgtCounty_Bnds, TgtCounty_Name = parameter_fn(county_dict, TgtCounty_GEOID, start_date_str)
    valid_dates, TgtCounty_Bnds, TgtCounty_Name, cPara_coefArr = parameter_fn(GEOID, TgtCounty_GEOID, start_date_str)
  
    #print("TgtCounty_Bnds", TgtCounty_Bnds)
    print("TgtCounty_Name", TgtCounty_Name)
    print("valid_dates", valid_dates)

    ET_retrieval_county_fn(valid_dates, TgtCounty_Bnds, TgtCounty_Name, cPara_coefArr)


In [ ]:
#@title Final call to run all functions

# Dictionary of targeted counties
county_dict = {# '08031': 'Denver',
'08039': 'Elbert',
# '08047': 'Gilpin',
# '08081': 'Moffat',
# '08095': 'Phillips',
# '08111': 'San_Juan',
}

in_year = 2000
ET_retrieval_wrapper(county_dict, in_year)

14
TgtCounty_Name Elbert
valid_dates ['2000-01-19', '2000-05-10', '2000-06-02', '2000-06-18', '2000-07-04', '2000-07-13', '2000-07-20', '2000-08-05', '2000-08-21', '2000-09-15', '2000-10-08', '2000-10-17', '2000-10-24', '2000-11-25']


# Debug

In [ ]:
#ARD ETa band mean
Date_st = ee.Date('2003-05-24')
Date_ed = Date_st.advance(1, 'day')

TgtCounty_Bnds = Counties.filter(ee.Filter.eq('GEOID', '08081'))

def maskLandsatARD_ET(image): 
  fillBitMask = 1
  clearBitMask = 1 << 1
  cloudBitMask = 1 << 5
  # var cloudConfBit6Mask = 1 << 6;
  # var cloudConfBit7Mask = 1 << 7;
  # var cirrusConfBit8Mask = 1 << 8;
  # var cirrusConfBit9Mask = 1 << 9;
  
  ETa_name = 'b1' #Et ... TODO
  ETf_name = 'b2'
  ETun_name = 'b3'
  pixelqa_name = 'b4'
  
  ETqa_threshold = 1.0 #1-2 mm
  
  #  get the pixel QA band
  #  pixelqa_name: 'b2'
  pixelqa = image.select(pixelqa_name) #
  
  #  get the STQA band
  # lstqa = image.select(lstqa_name)
  # stqa_scale = ee.Number(image.get(lstqa_scale_name))
  
  mask = pixelqa.bitwiseAnd(fillBitMask).eq(0)  \
    .And(pixelqa.bitwiseAnd(clearBitMask).neq(0)) \
    .And(pixelqa.bitwiseAnd(cloudBitMask).eq(0))
        #.and(pixelqa.bitwiseAnd(cloudConfBit7Mask).eq(0))
        #.and(pixelqa.bitwiseAnd(cirrusConfBit9Mask).eq(0))
        # .and(lstqa.multiply(stqa_scale).lt(lstqa_threshold))
        
        
  #  Return the masked image (excluding the PIXELQA and STQA layers)
  return image.updateMask(mask).select(ETa_name)
      #  .updateMask(mask_1);// [lst_name, pixelqa_name, lstqa_name]

ARD_ET = ee.Image(ARD_ET_5 \
  .filterDate(Date_st.format('yyyy-MM-dd'), Date_ed.format('yyyy-MM-dd')) \
  .filterBounds(TgtCounty_Bnds) \
  .map(maskLandsatARD_ET) \
  .mosaic())  \
  .multiply(0.001)  \
  .clip(TgtCounty_Bnds)

ARD_ET_mean = ARD_ET.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100)
print(ARD_ET_mean.getInfo())


{'b1': 1.0496155825616242}


In [ ]:
counties = ['Denver',
            #'Gilpin',
            'Elbert',
            'Moffat',
            'Phillips',
            'San_Juan']
for i in counties:
  print(i)
  # !earthengine ls projects/earthengine-legacy/assets/projects/ColoradoView/ET_Retrieval_c97/{i}/Ls5
# import numpy as np
# np.arange(2002, 2012, 2)
# lat = 40.5
# print(math.acos(math.tan(lat) * -1))
# print(1.0e-5 * 1000)
# print(0.409 * math.sin(2*math.pi/365 * 216 - 1.39))


Denver
Elbert
Moffat
Phillips
San_Juan


In [ ]:
#@title Loop for output values to compare with GEE

# test_date = ['2018-06-11']
# for x in test_date:
#   test_day = ee.Date(x)
#   PET_value = potential_ET_fn(test_day)
#   ETa_scene = Actual_ET_Fn(test_day, PET_value)

#   PET_mean = PET_value.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
#   print(PET_mean)
#   ETa_mean = ETa_scene.reduceRegion(ee.Reducer.mean(), TgtCounty_Bnds, 100).getInfo()
#   print(ETa_mean)

In [ ]:
#@title Remove files in GEE asset folder

assets = [
'projects/earthengine-legacy/assets/projects/ColoradoView/ET_Retrieval/Weld_Ls7'
]

for i in assets:
  print(i)
  !earthengine rm {i}


In [ ]:
counties = [# 'Adams',
# 'Alamosa',
# 'Arapahoe',
# 'Archuleta',
# 'Baca',
# 'Bent',
# 'Boulder',
# 'Broomfield',
# 'Chaffee',
# 'Cheyenne',
# 'Clear_Creek',
# 'Conejos',
# 'Costilla',
# 'Crowley',
# 'Custer',
# 'Delta',
'Denver',
# 'Dolores',
# 'Douglas',
# 'Eagle',
# 'El_Paso',
'Elbert',
# 'Fremont',
# 'Garfield',
'Gilpin',
# 'Grand',
# 'Gunnison',
# 'Hinsdale',
# 'Huerfano',
# 'Jackson',
# 'Jefferson',
# 'Kiowa',
# 'Kit_Carson',
# 'La_Plata',
# 'Lake',
# 'Larimer',
# 'Las_Animas',
# 'Lincoln',
# 'Logan',
# 'Mesa',
# 'Mineral',
'Moffat',
# 'Montezuma',
# 'Montrose',
# 'Morgan',
# 'Otero',
# 'Ouray',
# 'Park',
'Phillips',
# 'Pitkin',
# 'Prowers',
# 'Pueblo',
# 'Rio_Blanco',
# 'Rio_Grande',
# 'Routt',
# 'Saguache',
'San_Juan',
# 'San_Miguel',
# 'Sedgwick',
# 'Summit',
# 'Teller',
# 'Washington',
# 'Weld',
# 'Yuma'
]

for i in counties:
  # print(i)

  # !earthengine create folder projects/ColoradoView/ET_Retrieval/{i}/Ls5
  # !earthengine create folder projects/ColoradoView/ET_Retrieval/{i}/Ls7
  # !earthengine create folder projects/ColoradoView/ET_Retrieval/{i}/Ls8
  !earthengine ls projects/earthengine-legacy/assets/projects/ColoradoView/ET_Retrieval_c97/{i}/Ls5


In [ ]:
CO_cPara_DF = pd.read_pickle('/gdrive/MyDrive/CO_cPara_Crop_DF.pkl')
SeasonalCO_cPara1 = CO_cPara_DF.groupby(['Year','Season']).agg({'mean':['mean']}).reset_index()
# SeasonalCO_cPara.groupby('Season').agg({'mean mean': ['mean']}).reset_index()
SeasonalCO_cPara1.columns = [' '.join(col).strip() for col in SeasonalCO_cPara1.columns.values]
SeasonalCO_cPara = SeasonalCO_cPara1.groupby('Season').agg({'mean mean': ['mean']}).reset_index()
SeasonalCO_cPara.columns = [' '.join(col).strip() for col in SeasonalCO_cPara.columns.values]
SeasonalCO_cPara = SeasonalCO_cPara.rename(columns={'mean mean mean':'mean'})
SeasonalCO_cPara = SeasonalCO_cPara.set_index('Season')
Seasonal_cPara = SeasonalCO_cPara.to_dict('dict')
print(Seasonal_cPara)

valid_ds = ['2004-02-15', '2004-05-15', '2004-01-15', '2004-11-15']
cPara_Arr = []

for x in valid_ds:
  dt_conversion = datetime.datetime.strptime(x, '%Y-%m-%d')
  month_date = dt_conversion.month
  if 1 <= month_date <= 3:
    season = 'Winter'
    cPara = Seasonal_cPara['mean'][season]
  elif 3 < month_date <= 6:
    season = 'Spring'
    cPara = Seasonal_cPara['mean'][season]
  elif 6 < month_date <= 9:
    season = 'Summer'
    cPara = Seasonal_cPara['mean'][season]
  else:
    season = 'Fall'  
    cPara = Seasonal_cPara['mean'][season]
    # print(cPara)
    # print(type(cPara))
  cPara_Arr.append(cPara)

print(cPara_Arr)

{'mean': {'Fall': 0.996239931362002, 'Spring': 1.0020484104730463, 'Summer': 0.9890857449012788, 'Winter': 1.0019607117420657}}
[1.0019607117420657, 1.0020484104730463, 1.0019607117420657, 0.996239931362002]


In [ ]:
# county_dict = {'08001': 'Adams',
# '08003': 'Alamosa',
# '08005': 'Arapahoe',
# '08007': 'Archuleta',
# '08009': 'Baca',
# '08011': 'Bent',
# '08013': 'Boulder',
# '08015': 'Chaffee',
# '08017': 'Cheyenne',
# '08019': 'Clear_Creek',
# '08021': 'Conejos',
# '08023': 'Costilla',
# '08025': 'Crowley',
# '08027': 'Custer',
# '08029': 'Delta',
# '08031': 'Denver',
# '08033': 'Dolores',
# '08035': 'Douglas',
# '08037': 'Eagle',
# '08041': 'El_Paso',
# '08039': 'Elbert',
# '08043': 'Fremont',
# '08045': 'Garfield',
# '08047': 'Gilpin',
# '08049': 'Grand',
# '08051': 'Gunnison',
# '08053': 'Hinsdale',
# '08055': 'Huerfano',
# '08057': 'Jackson',
# '08059': 'Jefferson',
# '08061': 'Kiowa',
# '08063': 'Kit_Carson',
# '08067': 'La_Plata',
# '08065': 'Lake',
# '08069': 'Larimer',
# '08071': 'Las_Animas',
# '08073': 'Lincoln',
# '08075': 'Logan',
# '08077': 'Mesa',
# '08079': 'Mineral',
# '08081': 'Moffat',
# '08083': 'Montezuma',
# '08085': 'Montrose',
# '08087': 'Morgan',
# '08089': 'Otero',
# '08091': 'Ouray',
# '08093': 'Park',
# '08095': 'Phillips',
# '08097': 'Pitkin',
# '08099': 'Prowers',
# '08101': 'Pueblo',
# '08103': 'Rio_Blanco',
# '08105': 'Rio_Grande',
# '08107': 'Routt',
# '08109': 'Saguache',
# '08111': 'San_Juan',
# '08113': 'San_Miguel',
# '08115': 'Sedgwick',
# '08117': 'Summit',
# '08119': 'Teller',
# '08121': 'Washington',
# '08123': 'Weld',
# '08125': 'Yuma'
# }

# mean    1.002048
# Name: Spring, dtype: float64
# mean    1.001961
# Name: Winter, dtype: float64
# mean    0.99624
# Name: Fall, dtype: float64
# mean    0.989086
# Name: Summer, dtype: float64

moffat_mo_cPara = {('mean', 'mean'): {
  1: 0.9776521179661066,
  2: 0.9794218145513996,
  3: 0.9912462927803484,
  4: 1.012118814985312,
  5: 1.0064337392680458,
  6: 1.0034009170636713,
  7: 0.9922805196758296,
  8: 1.0047740820772288,
  9: 0.9965163265707929,
  10: 1.0117940268127896,
  11: 0.9839506809286691,
  12: 0.9777214143383488}}

# print(type(moffat_mo_cPara))

# mean_index = ('mean', 'mean')

# cPara_CoefArr = []

# for x in valid_dates:
#   dt_conversion = datetime.strptime(x, '%Y-%m-%d')
#   month_date = dt_conversion.month
#   print(month_date)
#   cPara_monthMean = moffat_mo_cPara[mean_index][month_date]
#   print(cPara_monthMean)
  # cPara_CoefArr.append(cPara_monthMean)
  # toDO: implement test cPara for monthly mean whether it has a valid c para.

# !earthengine ls projects/ColoradoView/ET_Seasonal_cPara/Phillips/Ls5
this = np.arange(2000, 2012, 1)
for year in this:
  print(year)
# print(this)

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
